# Imports Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
from pathlib import Path

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

c:\Users\rodol\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [31]:
output_dir = Path("../outputs/eda_plots")

# Loading Dataset

In [4]:
datasets = {}
datasets['register'] = pd.read_parquet("../data/base_cadastral.parquet")
datasets['submission'] = pd.read_parquet("../data/base_submissao.parquet")
datasets['loan'] = pd.read_parquet("../data/historico_emprestimos.parquet")
datasets['installments'] = pd.read_parquet("../data/historico_parcelas.parquet")

In [5]:
for name, df in datasets.items():
    print(f"\n📁 {name.upper()} DATASET:")
    print(f"   Shape: {df.shape}")
    print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"   Columns: {list(df.columns)}")
    print(f"   Data types:")
    for col, dtype in df.dtypes.items():
        print(f"     {col}: {dtype}")


📁 REGISTER DATASET:
   Shape: (40000, 16)
   Memory usage: 26.81 MB
   Columns: ['id_cliente', 'sexo', 'data_nascimento', 'qtd_filhos', 'qtd_membros_familia', 'renda_anual', 'tipo_renda', 'ocupacao', 'tipo_organizacao', 'nivel_educacao', 'estado_civil', 'tipo_moradia', 'possui_carro', 'possui_imovel', 'nota_regiao_cliente', 'nota_regiao_cliente_cidade']
   Data types:
     id_cliente: int64
     sexo: object
     data_nascimento: object
     qtd_filhos: int64
     qtd_membros_familia: float64
     renda_anual: float64
     tipo_renda: object
     ocupacao: object
     tipo_organizacao: object
     nivel_educacao: object
     estado_civil: object
     tipo_moradia: object
     possui_carro: object
     possui_imovel: object
     nota_regiao_cliente: int64
     nota_regiao_cliente_cidade: int64

📁 SUBMISSION DATASET:
   Shape: (40000, 8)
   Memory usage: 9.10 MB
   Columns: ['id_cliente', 'data_solicitacao', 'dia_semana_solicitacao', 'hora_solicitacao', 'tipo_contrato', 'valor_credito',

- id_cliente and id_contrato need to be converted to object
- sexo could be boolean
- data_nascimento, data_solicitacao, data_decisao.data_liberação, data_primeiro_vencimento, data-ultimo_vencimento_original, data_ultimo_vencimento, data_encerramento need to be converted to datetime
- possui_carro, possui_imovel,flag_ultima_solicitacao_contrato, flag_ultima_solicitacao_dia could be boolean
- versao_parcela maybe be converted to object

# Data Quality Analysis

In [7]:
for name, df in datasets.items():

    print(f"\n🔍 {name.upper()} - Data Quality Report:")
    
    # Missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing_Count': missing.values,
        'Missing_Percentage': missing_pct.values
    }).sort_values('Missing_Percentage', ascending=False)
    
    print("   Missing Values:")
    for _, row in missing_df[missing_df['Missing_Percentage'] > 0].iterrows():
        print(f"     {row['Column']}: {row['Missing_Count']} ({row['Missing_Percentage']:.1f}%)")
    
    # Duplicates
    duplicates = df.duplicated().sum()
    print(f"   Duplicate rows: {duplicates} ({duplicates/len(df)*100:.2f}%)")
    
    # Unique values for categorical columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    print("   Categorical columns unique values:")
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"     {col}: {unique_count} unique values")


🔍 REGISTER - Data Quality Report:
   Missing Values:
     ocupacao: 12676 (31.7%)
   Duplicate rows: 0 (0.00%)
   Categorical columns unique values:
     sexo: 2 unique values
     data_nascimento: 14943 unique values
     tipo_renda: 8 unique values
     ocupacao: 18 unique values
     tipo_organizacao: 58 unique values
     nivel_educacao: 5 unique values
     estado_civil: 5 unique values
     tipo_moradia: 6 unique values
     possui_carro: 2 unique values
     possui_imovel: 2 unique values

🔍 SUBMISSION - Data Quality Report:
   Missing Values:
     valor_bem: 24 (0.1%)
     valor_parcela: 4 (0.0%)
   Duplicate rows: 0 (0.00%)
   Categorical columns unique values:
     data_solicitacao: 21 unique values
     dia_semana_solicitacao: 7 unique values
     tipo_contrato: 2 unique values

🔍 LOAN - Data Quality Report:
   Missing Values:
     taxa_juros_promocional: 186262 (99.7%)
     taxa_juros_padrao: 186262 (99.7%)
     data_liberacao: 179790 (96.2%)
     data_encerramento: 100803

- tipo_contrato only have 2 unique values, could be boolean
- high percentage og missing values in LOAN dataset

# Statistical Summary

In [ ]:
for name, df in datasets.items():
    print(f"\n📊 {name.upper()} - Statistical Summary:")
    
    # Numerical columns summary
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    if len(numerical_cols) > 0:
        print("   Numerical columns:")
        summary = df[numerical_cols].describe()
        print(summary.round(2))
    
    # Categorical columns summary
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    if len(categorical_cols) > 0:
        print("   Categorical columns top values:")
        for col in categorical_cols:
            value_counts = df[col].value_counts().head(5)
            print(f"     {col}:")
            for val, count in value_counts.items():
                print(f"       {val}: {count} ({count/len(df)*100:.1f}%)")


📊 REGISTER - Statistical Summary:
   Numerical columns:
       id_cliente  qtd_filhos  qtd_membros_familia   renda_anual  \
count    40000.00    40000.00             40000.00  4.000000e+04   
mean    278414.66        0.41                 2.15  1.728539e+05   
std     102575.20        0.72                 0.90  6.025102e+05   
min     100023.00        0.00                 1.00  2.610000e+04   
25%     190486.25        0.00                 2.00  1.125000e+05   
50%     277796.50        0.00                 2.00  1.485000e+05   
75%     367312.25        1.00                 3.00  2.025000e+05   
max     456248.00       14.00                15.00  1.170000e+08   

       nota_regiao_cliente  nota_regiao_cliente_cidade  
count             40000.00                    40000.00  
mean                  2.05                        2.03  
std                   0.51                        0.50  
min                   1.00                        1.00  
25%                   2.00                   

- nota can be 1,2 or 3

# Customer Demographics Analysis

In [23]:
df = datasets['register']
max_submission_date = datasets['submission']['data_solicitacao'].max()

# Age analysis
df['idade'] = (pd.Timestamp(max_submission_date) - pd.to_datetime(df['data_nascimento'])).dt.days / 365.25

print(f"\n📊 Age Statistics:")
print(f"   Mean age: {df['idade'].mean():.1f} years")
print(f"   Median age: {df['idade'].median():.1f} years")
print(f"   Age range: {df['idade'].min():.1f} - {df['idade'].max():.1f} years")

# Gender distribution
print(f"\n👫 Gender Distribution:")
gender_dist = df['sexo'].value_counts()
for gender, count in gender_dist.items():
    print(f"   {gender}: {count} ({count/len(df)*100:.1f}%)")

# Income analysis
print(f"\n💰 Income Analysis:")
print(f"   Mean annual income: R$ {df['renda_anual'].mean():,.2f}")
print(f"   Median annual income: R$ {df['renda_anual'].median():,.2f}")
print(f"   Income range: R$ {df['renda_anual'].min():,.2f} - R$ {df['renda_anual'].max():,.2f}")

# Education level
print(f"\n🎓 Education Level Distribution:")
edu_dist = df['nivel_educacao'].value_counts()
for edu, count in edu_dist.items():
    print(f"   {edu}: {count} ({count/len(df)*100:.1f}%)")

# Family composition
print(f"\n👨‍👩‍👧‍👦 Family Composition:")
print(f"   Average family members: {df['qtd_membros_familia'].mean():.1f}")
print(f"   Average children: {df['qtd_filhos'].mean():.1f}")

# Asset ownership
print(f"\n🏠 Asset Ownership:")
car_owners = df['possui_carro'].value_counts()
house_owners = df['possui_imovel'].value_counts()

print(f"   Car owners: {car_owners.get('Y', 0)} ({car_owners.get('Y', 0)/len(df)*100:.1f}%)")
print(f"   House owners: {house_owners.get('Y', 0)} ({house_owners.get('Y', 0)/len(df)*100:.1f}%)")


📊 Age Statistics:
   Mean age: 44.0 years
   Median age: 43.2 years
   Age range: 21.0 - 69.0 years

👫 Gender Distribution:
   F: 26273 (65.7%)
   M: 13727 (34.3%)

💰 Income Analysis:
   Mean annual income: R$ 172,853.87
   Median annual income: R$ 148,500.00
   Income range: R$ 26,100.00 - R$ 117,000,000.00

🎓 Education Level Distribution:
   Secondary / secondary special: 28513 (71.3%)
   Higher education: 9633 (24.1%)
   Incomplete higher: 1366 (3.4%)
   Lower secondary: 463 (1.2%)
   Academic degree: 25 (0.1%)

👨‍👩‍👧‍👦 Family Composition:
   Average family members: 2.2
   Average children: 0.4

🏠 Asset Ownership:
   Car owners: 13625 (34.1%)
   House owners: 27686 (69.2%)


# Credit Application Analysis

In [15]:
df = datasets['submission']

# Application timing analysis
df['data_solicitacao'] = pd.to_datetime(df['data_solicitacao'])
df['hora_solicitacao'] = pd.to_datetime(df['hora_solicitacao'])

print(f"\n📅 Application Timing:")
print(f"   Date range: {df['data_solicitacao'].min().date()} to {df['data_solicitacao'].max().date()}")
print(f"   Total applications: {len(df)}")

# Day of week analysis
print(f"\n📅 Day of Week Distribution:")
dow_dist = df['dia_semana_solicitacao'].value_counts()
for day, count in dow_dist.items():
    print(f"   {day}: {count} ({count/len(df)*100:.1f}%)")

# Hour analysis
df['hour'] = df['hora_solicitacao'].dt.hour
print(f"\n🕐 Hour of Day Distribution:")
hour_dist = df['hour'].value_counts().sort_index()
for hour, count in hour_dist.items():
    print(f"   {hour:02d}:00: {count} ({count/len(df)*100:.1f}%)")

# Credit amount analysis
print(f"\n💰 Credit Amount Analysis:")
print(f"   Mean credit amount: R$ {df['valor_credito'].mean():,.2f}")
print(f"   Median credit amount: R$ {df['valor_credito'].median():,.2f}")
print(f"   Amount range: R$ {df['valor_credito'].min():,.2f} - R$ {df['valor_credito'].max():,.2f}")

# Contract type analysis
print(f"\n📋 Contract Type Distribution:")
contract_dist = df['tipo_contrato'].value_counts()
for contract, count in contract_dist.items():
    print(f"   {contract}: {count} ({count/len(df)*100:.1f}%)")


📅 Application Timing:
   Date range: 2025-02-04 to 2025-02-24
   Total applications: 40000

📅 Day of Week Distribution:
   TUESDAY: 7231 (18.1%)
   WEDNESDAY: 6805 (17.0%)
   THURSDAY: 6698 (16.7%)
   MONDAY: 6543 (16.4%)
   FRIDAY: 6384 (16.0%)
   SATURDAY: 4334 (10.8%)
   SUNDAY: 2005 (5.0%)

🕐 Hour of Day Distribution:
   00:00: 40000 (100.0%)

💰 Credit Amount Analysis:
   Mean credit amount: R$ 586,259.67
   Median credit amount: R$ 502,497.00
   Amount range: R$ 45,000.00 - R$ 4,050,000.00

📋 Contract Type Distribution:
   Cash loans: 36599 (91.5%)
   Revolving loans: 3401 (8.5%)


# Loan History Analysis

In [26]:
df = datasets['loan']
        
# Contract status analysis
print(f"\n📋 Contract Status Distribution:")
status_dist = df['status_contrato'].value_counts()
for status, count in status_dist.items():
    print(f"   {status}: {count} ({count/len(df)*100:.1f}%)")

# Contract type analysis
print(f"\n📋 Contract Type Distribution:")
contract_dist = df['tipo_contrato'].value_counts()
for contract, count in contract_dist.items():
    print(f"   {contract}: {count} ({count/len(df)*100:.1f}%)")

# Financial analysis
print(f"\n💰 Financial Analysis:")
print(f"   Mean requested amount: R$ {df['valor_solicitado'].mean():,.2f}")
print(f"   Mean approved amount: R$ {df['valor_credito'].mean():,.2f}")
print(f"   Mean down payment: R$ {df['valor_entrada'].mean():,.2f}")
print(f"   Mean installment: R$ {df['valor_parcela'].mean():,.2f}")

# Interest rate analysis
print(f"\n📈 Interest Rate Analysis:")
print(f"   Mean standard rate: {df['taxa_juros_padrao'].mean():.2f}%")
print(f"   Mean promotional rate: {df['taxa_juros_promocional'].mean():.2f}%")

# Customer type analysis
print(f"\n👥 Customer Type Distribution:")
customer_dist = df['tipo_cliente'].value_counts()
for customer, count in customer_dist.items():
    print(f"   {customer}: {count} ({count/len(df)*100:.1f}%)")


📋 Contract Status Distribution:
   Approved: 116182 (62.2%)
   Canceled: 35767 (19.1%)
   Refused: 32108 (17.2%)
   Unused offer: 2833 (1.5%)

📋 Contract Type Distribution:
   Cash loans: 83546 (44.7%)
   Consumer loans: 81684 (43.7%)
   Revolving loans: 21628 (11.6%)
   XNA: 32 (0.0%)

💰 Financial Analysis:
   Mean requested amount: R$ 174,010.34
   Mean approved amount: R$ 194,709.44
   Mean down payment: R$ 6,734.05
   Mean installment: R$ 15,904.69

📈 Interest Rate Analysis:
   Mean standard rate: 0.18%
   Mean promotional rate: 0.77%

👥 Customer Type Distribution:
   Repeater: 137719 (73.7%)
   New: 33828 (18.1%)
   Refreshed: 15126 (8.1%)
   XNA: 217 (0.1%)


# Payment History Analysis

In [27]:
df = datasets['installments']
        
# Payment timing analysis
df['data_prevista_pagamento'] = pd.to_datetime(df['data_prevista_pagamento'])
df['data_real_pagamento'] = pd.to_datetime(df['data_real_pagamento'])

# Calculate payment delays
df['payment_delay'] = (df['data_real_pagamento'] - df['data_prevista_pagamento']).dt.days

print(f"\n⏰ Payment Timing Analysis:")
print(f"   Total installments: {len(df)}")
print(f"   Mean payment delay: {df['payment_delay'].mean():.1f} days")
print(f"   Median payment delay: {df['payment_delay'].median():.1f} days")

# On-time vs late payments
on_time = (df['payment_delay'] <= 0).sum()
late = (df['payment_delay'] > 0).sum()
print(f"   On-time payments: {on_time} ({on_time/len(df)*100:.1f}%)")
print(f"   Late payments: {late} ({late/len(df)*100:.1f}%)")

# Payment amount analysis
print(f"\n💰 Payment Amount Analysis:")
print(f"   Mean expected amount: R$ {df['valor_previsto_parcela'].mean():,.2f}")
print(f"   Mean paid amount: R$ {df['valor_pago_parcela'].mean():,.2f}")

# Partial payments
partial_payments = (df['valor_pago_parcela'] < df['valor_previsto_parcela']).sum()
print(f"   Partial payments: {partial_payments} ({partial_payments/len(df)*100:.1f}%)")


⏰ Payment Timing Analysis:
   Total installments: 1390978
   Mean payment delay: -9.4 days
   Median payment delay: -7.0 days
   On-time payments: 1269751 (91.3%)
   Late payments: 120888 (8.7%)

💰 Payment Amount Analysis:
   Mean expected amount: R$ 18,145.87
   Mean paid amount: R$ 18,353.37
   Partial payments: 138800 (10.0%)


# Visualizations

## Customer Demographic

In [33]:
df = datasets['register']
df['idade'] = (pd.Timestamp.now() - pd.to_datetime(df['data_nascimento'])).dt.days / 365.25

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Customer Demographics Analysis', fontsize=16, fontweight='bold')

# Age distribution
axes[0, 0].hist(df['idade'].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Age Distribution')
axes[0, 0].set_xlabel('Age (years)')
axes[0, 0].set_ylabel('Frequency')

# Gender distribution
gender_counts = df['sexo'].value_counts()
axes[0, 1].pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('Gender Distribution')

# Income distribution
axes[0, 2].hist(df['renda_anual'].dropna(), bins=100, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 2].set_title('Annual Income Distribution')
axes[0, 2].set_xlabel('Annual Income (R$)')
axes[0, 2].set_ylabel('Frequency')

# Education level
edu_counts = df['nivel_educacao'].value_counts()
axes[1, 0].bar(range(len(edu_counts)), edu_counts.values, color='lightcoral')
axes[1, 0].set_title('Education Level Distribution')
axes[1, 0].set_xlabel('Education Level')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_xticks(range(len(edu_counts)))
axes[1, 0].set_xticklabels(edu_counts.index, rotation=45)

# Family size
axes[1, 1].hist(df['qtd_membros_familia'].dropna(), bins=15, alpha=0.7, color='gold', edgecolor='black')
axes[1, 1].set_title('Family Size Distribution')
axes[1, 1].set_xlabel('Family Members')
axes[1, 1].set_ylabel('Frequency')

# Asset ownership
car_owners = df['possui_carro'].value_counts()
house_owners = df['possui_imovel'].value_counts()

x = np.arange(2)
width = 0.35

axes[1, 2].bar(x - width/2, [car_owners.get('Y', 0), car_owners.get('N', 0)], 
                width, label='Car', color='lightblue')
axes[1, 2].bar(x + width/2, [house_owners.get('Y', 0), house_owners.get('N', 0)], 
                width, label='House', color='lightgreen')
axes[1, 2].set_title('Asset Ownership')
axes[1, 2].set_xlabel('Ownership')
axes[1, 2].set_ylabel('Count')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(['Yes', 'No'])
axes[1, 2].legend()

plt.tight_layout()
plt.savefig(output_dir / 'customer_demographics.png', dpi=300, bbox_inches='tight')
plt.close()

- income distribution right skewed


## Credit Applications

In [34]:
df = datasets['submission']
df['data_solicitacao'] = pd.to_datetime(df['data_solicitacao'])
df['hora_solicitacao'] = pd.to_datetime(df['hora_solicitacao'])
df['hour'] = df['hora_solicitacao'].dt.hour

fig, axes = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('Credit Application Patterns', fontsize=16, fontweight='bold')

# Applications over time
daily_apps = df.groupby(df['data_solicitacao'].dt.date).size()
axes[0, 0].plot(daily_apps.index, daily_apps.values, linewidth=2, color='blue')
axes[0, 0].set_title('Daily Applications Over Time')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Number of Applications')
axes[0, 0].tick_params(axis='x', rotation=45)

# Day of week distribution
dow_counts = df['dia_semana_solicitacao'].value_counts()
axes[0, 1].bar(range(len(dow_counts)), dow_counts.values, color='orange')
axes[0, 1].set_title('Applications by Day of Week')
axes[0, 1].set_xlabel('Day of Week')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_xticks(range(len(dow_counts)))
axes[0, 1].set_xticklabels(dow_counts.index, rotation=45)

# Hour of day distribution
hour_counts = df['hour'].value_counts().sort_index()
axes[1, 0].bar(hour_counts.index, hour_counts.values, color='green')
axes[1, 0].set_title('Applications by Hour of Day')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('Count')

# Credit amount distribution
axes[1, 1].hist(df['valor_credito'].dropna(), bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1, 1].set_title('Credit Amount Distribution')
axes[1, 1].set_xlabel('Credit Amount (R$)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig(output_dir / 'credit_applications.png', dpi=300, bbox_inches='tight')
plt.close()

- few submission in sunday

## Loans

In [36]:
df = datasets['loan']
        
fig, axes = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('Loan Performance Analysis', fontsize=16, fontweight='bold')

# Contract status distribution
status_counts = df['status_contrato'].value_counts()
axes[0, 0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title('Contract Status Distribution')

# Contract type distribution
contract_counts = df['tipo_contrato'].value_counts().head(10)
axes[0, 1].bar(range(len(contract_counts)), contract_counts.values, color='lightblue')
axes[0, 1].set_title('Top 10 Contract Types')
axes[0, 1].set_xlabel('Contract Type')
axes[0, 1].set_ylabel('Count')
axes[0, 1].set_xticks(range(len(contract_counts)))
axes[0, 1].set_xticklabels(contract_counts.index, rotation=45)

# Credit amount vs requested amount
axes[1, 0].scatter(df['valor_solicitado'], df['valor_credito'], alpha=0.6, color='red')
axes[1, 0].plot([df['valor_solicitado'].min(), df['valor_solicitado'].max()], 
                [df['valor_solicitado'].min(), df['valor_solicitado'].max()], 'k--', alpha=0.5)
axes[1, 0].set_title('Requested vs Approved Amount')
axes[1, 0].set_xlabel('Requested Amount (R$)')
axes[1, 0].set_ylabel('Approved Amount (R$)')

# Interest rate distribution
axes[1, 1].hist(df['taxa_juros_padrao'].dropna(), bins=30, alpha=0.7, color='gold', edgecolor='black')
axes[1, 1].set_title('Standard Interest Rate Distribution')
axes[1, 1].set_xlabel('Interest Rate (%)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig(output_dir / 'loan_performance.png', dpi=300, bbox_inches='tight')
plt.close()

## Installments

In [38]:
df = datasets['installments']
df['data_prevista_pagamento'] = pd.to_datetime(df['data_prevista_pagamento'])
df['data_real_pagamento'] = pd.to_datetime(df['data_real_pagamento'])
df['payment_delay'] = (df['data_real_pagamento'] - df['data_prevista_pagamento']).dt.days

fig, axes = plt.subplots(2, 2, figsize=(20, 12))
fig.suptitle('Payment Behavior Analysis', fontsize=16, fontweight='bold')

# Payment delay distribution
axes[0, 0].hist(df['payment_delay'].dropna(), bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 0].set_title('Payment Delay Distribution')
axes[0, 0].set_xlabel('Delay (days)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(x=0, color='red', linestyle='--', alpha=0.7, label='On-time')
axes[0, 0].legend()

# Payment amount comparison
axes[0, 1].scatter(df['valor_previsto_parcela'], df['valor_pago_parcela'], alpha=0.6, color='blue')
axes[0, 1].plot([df['valor_previsto_parcela'].min(), df['valor_previsto_parcela'].max()], 
                [df['valor_previsto_parcela'].min(), df['valor_previsto_parcela'].max()], 'k--', alpha=0.5)
axes[0, 1].set_title('Expected vs Actual Payment Amount')
axes[0, 1].set_xlabel('Expected Amount (R$)')
axes[0, 1].set_ylabel('Actual Amount (R$)')

# Payment timing over time
monthly_payments = df.groupby(df['data_real_pagamento'].dt.to_period('M')).size()
axes[1, 0].plot(range(len(monthly_payments)), monthly_payments.values, linewidth=2, color='green')
axes[1, 0].set_title('Monthly Payment Volume')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Number of Payments')

# On-time vs late payments
on_time = (df['payment_delay'] <= 0).sum()
late = (df['payment_delay'] > 0).sum()
axes[1, 1].pie([on_time, late], labels=['On-time', 'Late'], autopct='%1.1f%%', 
                colors=['lightgreen', 'lightcoral'], startangle=90)
axes[1, 1].set_title('Payment Punctuality')

plt.tight_layout()
plt.savefig(output_dir / 'payment_behavior.png', dpi=300, bbox_inches='tight')
plt.close()

# OTHERS

## Installments

In [ ]:
# datasets['installments'].iloc[:100000,:].to_excel("installments.xlsx")

In [11]:
datasets['installments'].query("valor_previsto_parcela != valor_pago_parcela")

,id_contrato,id_cliente,versao_parcela,numero_parcela,data_prevista_pagamento,data_real_pagamento,valor_previsto_parcela,valor_pago_parcela
6,2726751,182114,1.0,20,2022-03-15,2022-02-18,10770.750,499.500
9,1775456,134670,1.0,1,2022-06-13,2022-06-17,34703.820,32537.790
10,1915769,138380,1.0,17,2022-03-08,2022-03-15,16729.155,15592.365
11,2623487,174960,1.0,10,2022-06-17,2022-06-21,11463.975,11445.750
12,2127195,110953,1.0,8,2021-07-26,2021-08-02,54752.850,54459.675
...,...,...,...,...,...,...,...,...
1390943,2384200,431712,2.0,20,2022-09-21,2022-09-11,15488.325,1858.860
1390960,2206019,408583,3.0,12,2023-11-14,2023-11-18,34120.170,22780.350
1390975,1635645,417079,1.0,1,2019-01-09,2019-01-15,9914.850,9010.485
1390976,2246041,430618,0.0,40,2021-04-04,None,67.500,NaN


In [12]:
datasets['installments'].query("valor_pago_parcela == 0")

,id_contrato,id_cliente,versao_parcela,numero_parcela,data_prevista_pagamento,data_real_pagamento,valor_previsto_parcela,valor_pago_parcela
5178,2254262,197304,0.0,17,2018-09-20,2018-08-24,9000.0,0.0
10900,2116229,195907,0.0,13,2018-08-23,2018-07-27,9000.0,0.0
20110,1298920,103055,0.0,11,2018-05-31,2018-05-01,5625.0,0.0
21840,2052166,154455,0.0,27,2019-02-05,2019-01-09,6750.0,0.0
48693,1130681,137552,0.0,20,2019-03-19,2019-02-22,5625.0,0.0
...,...,...,...,...,...,...,...,...
1329909,2605243,422585,0.0,8,2018-12-20,2018-11-21,9000.0,0.0
1366840,2076417,442825,0.0,7,2019-01-27,2018-12-28,6750.0,0.0
1378518,2652019,445660,0.0,3,2019-03-10,2019-02-19,9000.0,0.0
1384823,1646712,405914,0.0,7,2018-11-29,2018-10-30,4500.0,0.0


## SUBMISSION

In [16]:
datasets['submission'].to_excel("submission.xlsx")

In [3]:
submission = pd.read_parquet("../data/base_submissao.parquet")
submission['hora_solicitacao'] 










0        12
1         9
2        10
3        11
4        12
         ..
39995    12
39996    10
39997    14
39998    11
39999    11
Name: hora_solicitacao, Length: 40000, dtype: int64